In [4]:
import cv2
import numpy as np
import os
from skimage.segmentation import clear_border

# Specify the path where images are stored
image_folder = '/content/drive/MyDrive/CTP/p1'

# Create a list to store the processed markers
processed_markers = []

# Iterate over all images in the folder (handling 136 images)
for image_name in os.listdir(image_folder):

    # Load the image
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)

    # Convert to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define adjusted HSV ranges for red
    lower_red1 = np.array([0, 30, 30])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 30, 30])
    upper_red2 = np.array([180, 255, 255])

    # Create a mask for red colors
    mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Clean the mask using morphological operations
    kernel = np.ones((5, 5), np.uint8)
    cleaned_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_OPEN, kernel, iterations=5)

    # Remove edge-touching regions
    cleaned_mask = clear_border(cleaned_mask)

    # Distance transform
    dist_transform = cv2.distanceTransform(cleaned_mask, cv2.DIST_L2, 3)

    # Threshold to capture more foreground (including shadow regions)
    _, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # Find unknown region
    sure_bg = cv2.dilate(cleaned_mask, kernel, iterations=6)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labeling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add 1 to all labels so background is not 0
    markers = markers + 1

    # Mark the unknown region with zero
    markers[unknown == 255] = 0

    # Apply watershed algorithm
    markers = cv2.watershed(image, markers)
    markers = clear_border(markers)

    # Store the markers for further processing
    processed_markers.append((image_name, markers))

    print(f"Processed {image_name}")

# Confirmation of output
print("All images processed and markers generated.")

Processed 30_01_2024_15.png
Processed 29_01_2024_21.png
Processed 27_01_2024_21.png
Processed 27_01_2024_07.png
Processed 29_01_2024_20.png
Processed 30_01_2024_07.png
Processed 30_01_2024_23.png
Processed 30_01_2024_16.png
Processed 30_01_2024_12.png
Processed 31_01_2024_00.png
Processed 31_01_2024_07.png
Processed 31_01_2024_02.png
Processed 30_01_2024_08.png
Processed 31_01_2024_03.png
Processed 31_01_2024_08.png
Processed 30_01_2024_10.png
Processed 30_01_2024_19.png
Processed 31_01_2024_10.png
Processed 30_01_2024_18.png
Processed 30_01_2024_22.png
Processed 31_01_2024_05.png
Processed 31_01_2024_12.png
Processed 30_01_2024_21.png
Processed 30_01_2024_13.png
Processed 31_01_2024_01.png
Processed 31_01_2024_04.png
Processed 31_01_2024_06.png
Processed 30_01_2024_11.png
Processed 31_01_2024_11.png
Processed 30_01_2024_17.png
Processed 31_01_2024_09.png
Processed 30_01_2024_06.png
Processed 30_01_2024_14.png
Processed 29_01_2024_07.png
Processed 28_01_2024_13.png
Processed 29_01_2024

In [5]:
import pandas as pd
import cv2
import numpy as np

# Initialize an empty list to store the rows for the dataframe
data_rows = []

# Iterate over all processed markers and extract cell attributes
for image_name, markers in processed_markers:
    # Get unique labels and exclude the first one (background or large region)
    unique_labels = np.unique(markers)
    valid_labels = unique_labels[(unique_labels > 1)]  # Exclude background (0) and border (-1)
    cell_number = 1
    # Iterate through valid labels
    for label in valid_labels:
        # Create a mask for the current label
        cell_mask = (markers == label).astype(np.uint8)
        if label == -1 or label == 1:  # Ignore boundary (-1) and background (1)
            continue

        # Find contours of the cell
        contours, _ = cv2.findContours(cell_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue

        cnt = contours[0]

        # Size (Area)
        area = cv2.contourArea(cnt)

        # Centroid
        M = cv2.moments(cnt)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
        else:
            cx, cy = None, None

        # Perimeter
        perimeter = cv2.arcLength(cnt, True)

        # Circularity
        if perimeter != 0:
            circularity = 4 * np.pi * area / (perimeter ** 2)
        else:
            circularity = 0

        # Bounding box
        x, y, w, h = cv2.boundingRect(cnt)

        # Aspect ratio
        aspect_ratio = w / h if h != 0 else 0

        # Solidity
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = area / hull_area if hull_area != 0 else 0

        # Append the data to the list as a row
        data_rows.append([image_name, cell_number, area, (cx, cy), perimeter, circularity, (x, y, w, h), aspect_ratio, solidity])
        cell_number += 1

# Create a pandas DataFrame from the collected data
df = pd.DataFrame(data_rows, columns=[
    'Image Name', 'Cell Number', 'Size (Area)', 'Centroid', 'Perimeter', 'Circularity',
    'Bounding Box', 'Aspect Ratio', 'Solidity'
])

# Save the dataframe to a CSV file
df.to_csv('cell_attributes.csv', index=False)

#Confirmation of all data processing finished
print("All images processed and data saved to CSV.")

All images processed and data saved to CSV.
